# Example
Comparison between swept wing by Bertin and pySailingVLM results. More information can be found at Aerodynamics for engineers John J.Bertin p 368 (Example 7.2).

In [1]:
# varaibles.py for jupyter
import os
import numpy as np
import time
from pySailingVLM.runner.container import Output, Rig, Conditions, Solver, MainSail, JibSail, Csys, Keel

half_wing_span = 0.5
sweep_angle_deg = 0.
chord_length = 0.2
AoA_deg = 4.
mgirths =  np.array([0.00, 1./8, 1./4, 1./2, 3./4, 7./8, 1.00])
mchords = np.array([chord_length]* len(mgirths))
jgirths = np.array([0.00, 1./4, 1./2, 3./4, 1.00])

out = Output(case_name='my_case_name',
             case_dir=os.path.abspath(''),
             name=os.path.join("results_example_jib_and_mainsail_vlm", time.strftime("%Y-%m-%d_%Hh%Mm%Ss")),
            file_name='my_fancy_results')

solver = Solver(n_spanwise=5,
                n_chordwise=3,
                interpolation_type='linear')

conditions = Conditions(leeway_deg=0.,    
                        heel_deg=0.,    
                        SOG_yacht=0.,  
                        tws_ref= 1.,     
                        alpha_true_wind_deg= AoA_deg, 
                        reference_water_level_for_wind_profile=-0.,
                        wind_exp_coeff=0.,
                        wind_reference_measurment_height=10.,
                        rho=1.,
                        wind_profile='flat',
                        roughness=0.05)

rig = Rig(main_sail_luff=half_wing_span / np.cos(np.deg2rad(sweep_angle_deg)),
          jib_luff=10.0,
          foretriangle_height=11.50,
          foretriangle_base=3.90,
          sheer_above_waterline=0.,
          boom_above_sheer=0.,
          rake_deg=90. + sweep_angle_deg,
          mast_LOA=0.,
          sails_def='main')

main = MainSail(centerline_twist_deg=np.array([0] * len(mgirths)), #0*mgirths,
                girths=mgirths,
                chords=mchords,
                camber= 0*np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]),
                camber_distance_from_luff=np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]))
              
jib = JibSail(centerline_twist_deg=0*(10+5)  + 0*15. * jgirths,
             girths=0*jgirths,
             chords=0* np.array([3.80, 2.98, 2.15, 1.33, 0.5]),
             camber=0*np.array([0.01, 0.01, 0.01, 0.01, 0.01]),
             camber_distance_from_luff=0*np.array([0.5, 0.5, 0.5, 0.5, 0.5]))

csys = Csys(reference_level_for_moments=np.array([0, 0, 0]))
keel = Keel(center_of_lateral_resistance_upright=np.array([0, 0, -1.0]))    

wind
<div class="alert alert-block alert-warning">
<b>Run cell below twice:</b> Run cell below twice before running other cells. Output below is expected to always appear at first time importing pySailingVLM:
<br><br>
    
<em>   
/home/user/miniconda3/envs/mgr_test/lib/python3.10/site-packages/numba/core/lowering.py:107: NumbaDebugInfoWarning: Could not find source for function: <function __numba_array_expr_0x7f01d6a1e9e0 at 0x7f01d6cfa680>. Debug line information may be inaccurate.warnings.warn(NumbaDebugInfoWarning(msg))
</em> 
<br>
</div>

In [2]:
import shutil
from pySailingVLM.rotations.csys_transformations import CSYS_transformations
from pySailingVLM.yacht_geometry.hull_geometry import HullGeometry
from pySailingVLM.results.save_utils import save_results_to_file
from pySailingVLM.solver.panels_plotter import display_panels_xyz_and_winds
from pySailingVLM.results.inviscid_flow import InviscidFlowResults
from pySailingVLM.solver.vlm import Vlm
from pySailingVLM.runner.sail import Wind, Sail
from pySailingVLM.solver.panels_plotter import plot_cp

In [3]:
import numpy as np
from pySailingVLM.solver.coefs import get_vlm_Cxyz

AoA_degs = np.linspace(0.001, 10., 20) # [4.0] #

C_results = []
cl_results = []
a_vlm_results = []
# for AoA_deg in AoA_degs:
#conditions.alpha_true_wind_deg=  AoA_deg
csys_transformations = CSYS_transformations(conditions.heel_deg, conditions.leeway_deg, v_from_original_xyz_2_reference_csys_xyz=csys.reference_level_for_moments)

w = Wind(conditions)
s = Sail(solver, rig, main, jib, csys_transformations)
sail_set = s.sail_set
myvlm = Vlm(sail_set.panels, solver.n_chordwise, solver.n_spanwise, conditions.rho, w.profile, sail_set.trailing_edge_info, sail_set.leading_edge_info)

height = 1.0
sails_Cxyz = myvlm.get_Cxyz(w, height)

    # enumerate through sails
    # in this example we have only main
for idx, Cxyz in enumerate(sails_Cxyz):
    a_vlm = Cxyz[1] / np.deg2rad(AoA_deg)
    C_results.append(Cxyz[1])
    a_vlm_results.append(a_vlm)


In [4]:
# results from Aerodynamics for engineers John J.Bertin p 368
C_bertin = 1.096 * np.pi * np.pi / 180.* AoA_degs

In [5]:
# import matplotlib.pyplot as plt
# import scienceplots # sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended cm-super

# # plt.style.use('science')
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)
# ax.plot(AoA_degs, C_results, color='tab:blue', label='pySailingVLM')
# ax.plot(AoA_degs, C_bertin, color='tab:orange', linestyle='dashed', label='Bertin')

# ax.set_xlabel(r"AoA [$^\circ$]")
# ax.set_ylabel(r"$C_{L}$")

# ax.set_title("Lift coefficient versus angle of attack.")
# ax.legend()
# plt.plot(AoA_degs, C_results)   
# #plt.savefig('bertin_1.png')

In [6]:
# import numpy as np
# from pySailingVLM.solver.coefs import get_vlm_Cxyz

# AoA_deg = 4.2
# C = []
# c_l = []
# a_vlm = []

# conditions.alpha_true_wind_deg = AoA_deg
# solver.n_spanwise = 4

# csys_transformations = CSYS_transformations(conditions.heel_deg, conditions.leeway_deg, v_from_original_xyz_2_reference_csys_xyz=csys.reference_level_for_moments)

# w = Wind(conditions)
# s = Sail(solver, rig, main, jib, csys_transformations)
# sail_set = s.sail_set
# myvlm = Vlm(sail_set.panels, solver.n_chordwise, solver.n_spanwise, conditions.rho, w.profile, sail_set.trailing_edge_info, sail_set.leading_edge_info)

# height = 1.0
# sails_Cxyz = myvlm.get_Cxyz(w, height)
 
# for idx, Cxyz in enumerate(sails_Cxyz):
#     a = Cxyz[1] / np.deg2rad(AoA_deg)
#     C.append(Cxyz[1])
#     a_vlm.append(a)

In [7]:
# def calculate_ydata(gamma, CL, cav, wind, height):
#     V = np.array(wind.profile.get_true_wind_speed_at_h(height))
#     return 2 * gamma / (np.linalg.norm(V) * cav * CL)

# N = int(sail_set.panels.shape[0] / 2)
# cav = chord_length
# y_data = np.array([calculate_ydata(myvlm.gamma_magnitude[i], C[0], cav, w, height) for i in range(N)])
# cp_y = np.split(myvlm.cp, 2)[0][:,2]
# x_data = 2 * cp_y / (half_wing_span * 2)

In [8]:
# g_good = np.array([0.0273, 0.0287, 0.0286, 0.0250])
# ydata_good = 2 * 4 * 2 * half_wing_span * g_good / (1.096 * cav)
# cp_y_good = np.array([0.0625, 0.1875, 0.3125, 0.4377])
# x_good = 2*cp_y_good / (2*half_wing_span)

In [9]:
# # importing matplotlib module
# from matplotlib import pyplot as plt
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)
# ax.plot(x_data, y_data, 'bo',label='pySailingVLM')
# ax.plot(x_good, ydata_good,'r+', label='Bertin')

# ax.set_xlabel(r"$\frac{2y}{b}$")
# ax.set_ylabel(r"$\frac{C_{l}}{C_{L}}$")
# ax.set_title("Spanwise lift distribution")
# ax.legend()   
# plt.savefig('bertin_2')

In [10]:
from pySailingVLM.solver.coefs import get_C
cl = get_C(myvlm.panels, myvlm.areas, myvlm.lift, myvlm.inlet_conditions.V_app_infs, myvlm.n_spanwise, myvlm.n_chordwise, myvlm.rho)
cd = get_C(myvlm.panels, myvlm.areas, myvlm.drag, myvlm.V_induced_at_cp, myvlm.n_spanwise, myvlm.n_chordwise, myvlm.rho)

In [11]:
#np.testing.assert_almost_equal(myvlm.inlet_conditions.V_app_infs,w.profile.get_true_wind_speed_at_h(1.0))
    

In [12]:
w.profile.get_true_wind_speed_at_h(1.0)

array([0.99756405, 0.06975647, 0.        ])

In [13]:
myvlm.inlet_conditions.V_app_infs

array([[0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.99756405, 0.06975647, 0.        ],
       [0.

In [14]:
myvlm.force

array([[-1.72092642e-04,  2.12051197e-03, -1.19413258e-20],
       [-1.66525455e-04,  2.08175912e-03, -1.15550247e-20],
       [-1.54142253e-04,  1.99224097e-03, -5.34788370e-21],
       [-1.31681669e-04,  1.81670623e-03, -9.13725128e-21],
       [-9.07493305e-05,  1.43966010e-03, -6.29699976e-21],
       [-1.12879368e-05,  8.23548283e-04,  0.00000000e+00],
       [-1.08340954e-05,  8.03577952e-04, -1.50353278e-21],
       [-9.80767593e-06,  7.56660716e-04,  0.00000000e+00],
       [-7.88710819e-06,  6.62586362e-04, -1.09455614e-21],
       [-4.28961984e-06,  4.64332190e-04,  0.00000000e+00],
       [-3.30374493e-06,  4.01403441e-04,  0.00000000e+00],
       [-3.12935818e-06,  3.89829640e-04,  0.00000000e+00],
       [-2.73723427e-06,  3.62732181e-04, -7.59735127e-22],
       [-2.03189027e-06,  3.09664382e-04,  0.00000000e+00],
       [-9.25948203e-07,  2.08369058e-04,  0.00000000e+00],
       [-9.07493305e-05,  1.43966010e-03,  6.29699976e-21],
       [-1.31681669e-04,  1.81670623e-03

In [15]:
myvlm.lift

array([[-1.48190933e-04,  2.11922907e-03, -1.02828116e-20],
       [-1.45484273e-04,  2.08052204e-03, -1.00949994e-20],
       [-1.39233194e-04,  1.99112744e-03, -4.83062180e-21],
       [-1.26978084e-04,  1.81587120e-03, -8.81087452e-21],
       [-1.00649099e-04,  1.43934917e-03, -6.98393418e-21],
       [-5.75443404e-05,  8.22922407e-04,  0.00000000e+00],
       [-5.61485418e-05,  8.02961557e-04, -7.79217549e-21],
       [-5.28701811e-05,  7.56078816e-04,  0.00000000e+00],
       [-4.62992965e-05,  6.62110788e-04, -6.42531813e-21],
       [-3.24559553e-05,  4.64141785e-04,  0.00000000e+00],
       [-2.80410366e-05,  4.01005506e-04,  0.00000000e+00],
       [-2.72311565e-05,  3.89423681e-04,  0.00000000e+00],
       [-2.53354996e-05,  3.62314524e-04, -7.03201375e-21],
       [-2.16247609e-05,  3.09248489e-04,  0.00000000e+00],
       [-1.45466144e-05,  2.08026278e-04,  0.00000000e+00],
       [-1.00649099e-04,  1.43934917e-03,  6.98393418e-21],
       [-1.26978084e-04,  1.81587120e-03

In [16]:
myvlm.drag

array([[-2.39017093e-05,  1.28289814e-06, -1.65851425e-21],
       [-2.10411819e-05,  1.23708016e-06, -1.46002529e-21],
       [-1.49090591e-05,  1.11353005e-06, -5.17261896e-22],
       [-4.70358490e-06,  8.35035246e-07, -3.26376766e-22],
       [ 9.89976822e-06,  3.10929681e-07,  6.86934414e-22],
       [ 4.62564036e-05,  6.25876672e-07, -0.00000000e+00],
       [ 4.53144463e-05,  6.16395491e-07,  6.28864271e-21],
       [ 4.30625052e-05,  5.81900464e-07, -0.00000000e+00],
       [ 3.84121883e-05,  4.75574367e-07,  5.33076199e-21],
       [ 2.81663355e-05,  1.90404735e-07, -0.00000000e+00],
       [ 2.47372917e-05,  3.97934515e-07, -0.00000000e+00],
       [ 2.41017983e-05,  4.05959092e-07, -0.00000000e+00],
       [ 2.25982653e-05,  4.17656945e-07,  6.27227863e-21],
       [ 1.95928707e-05,  4.15892963e-07, -0.00000000e+00],
       [ 1.36206662e-05,  3.42780738e-07, -0.00000000e+00],
       [ 9.89976822e-06,  3.10929681e-07, -6.86934414e-22],
       [-4.70358490e-06,  8.35035246e-07

In [17]:
myvlm.V_induced_at_ctrl

array([[ 6.75526678e-04, -6.97564737e-02, -4.63461069e-05],
       [ 6.71850500e-04, -6.97564737e-02, -1.47877596e-04],
       [ 6.50213928e-04, -6.97564737e-02, -2.78353169e-04],
       [ 5.63257262e-04, -6.97564737e-02, -4.61724835e-04],
       [ 2.72922540e-04, -6.97564737e-02, -6.80800068e-04],
       [ 8.60237576e-04, -6.97564737e-02, -3.82002885e-05],
       [ 8.84142070e-04, -6.97564737e-02, -1.24953931e-04],
       [ 9.26851049e-04, -6.97564737e-02, -2.49501468e-04],
       [ 9.58404043e-04, -6.97564737e-02, -4.68527633e-04],
       [ 7.81496669e-04, -6.97564737e-02, -9.39701319e-04],
       [ 1.11917355e-03, -6.97564737e-02,  1.01712147e-04],
       [ 1.19512698e-03, -6.97564737e-02,  3.36312435e-04],
       [ 1.37206647e-03, -6.97564737e-02,  6.88986834e-04],
       [ 1.72269791e-03, -6.97564737e-02,  1.36694721e-03],
       [ 2.49919269e-03, -6.97564737e-02,  3.23570473e-03],
       [ 2.72922540e-04, -6.97564737e-02,  6.80800068e-04],
       [ 5.63257262e-04, -6.97564737e-02

In [18]:
myvlm.V_induced_at_cp

array([[ 6.03886138e-04,  1.12510187e-02, -4.54729121e-05],
       [ 5.93152426e-04,  1.00887788e-02, -1.43248957e-04],
       [ 5.57883700e-04,  7.46950752e-03, -2.61785074e-04],
       [ 4.58733605e-04,  2.58395375e-03, -4.10133117e-04],
       [ 2.15494807e-04, -6.86119330e-03, -5.48087554e-04],
       [ 7.58701018e-04, -5.60729966e-02, -4.53766789e-05],
       [ 7.65782592e-04, -5.62966710e-02, -1.46675225e-04],
       [ 7.67754593e-04, -5.68163084e-02, -2.84683324e-04],
       [ 7.16520408e-04, -5.78734238e-02, -5.03044155e-04],
       [ 4.09230380e-04, -6.05369407e-02, -8.53342447e-04],
       [ 9.89924477e-04, -6.15378904e-02,  1.25904958e-20],
       [ 1.03992185e-03, -6.17401784e-02,  5.18164501e-20],
       [ 1.14993887e-03, -6.22200204e-02, -1.03629626e-19],
       [ 1.34157444e-03, -6.32020660e-02, -1.15814624e-20],
       [ 1.64376224e-03, -6.53162047e-02, -3.96733304e-21],
       [ 2.15494807e-04, -6.86119330e-03,  5.48087554e-04],
       [ 4.58733605e-04,  2.58395375e-03

In [19]:
np.linalg.norm(myvlm.lift, axis=1)

array([0.0021244 , 0.0020856 , 0.00199599, 0.00182031, 0.00144286,
       0.00082493, 0.00080492, 0.00075793, 0.00066373, 0.00046528,
       0.00040198, 0.00039037, 0.0003632 , 0.00031   , 0.00020853,
       0.00144286, 0.00182031, 0.00199599, 0.0020856 , 0.0021244 ,
       0.00046528, 0.00066373, 0.00075793, 0.00080492, 0.00082493,
       0.00020853, 0.00031   , 0.0003632 , 0.00039037, 0.00040198])

In [20]:
np.linalg.norm(myvlm.drag, axis=1)

array([2.39361136e-05, 2.10775166e-05, 1.49505850e-05, 4.77713248e-06,
       9.90464982e-06, 4.62606376e-05, 4.53186384e-05, 4.30664366e-05,
       3.84151322e-05, 2.81669790e-05, 2.47404922e-05, 2.41052169e-05,
       2.26021245e-05, 1.95972842e-05, 1.36249788e-05, 9.90464982e-06,
       4.77713248e-06, 1.49505850e-05, 2.10775166e-05, 2.39361136e-05,
       2.81669790e-05, 3.84151322e-05, 4.30664366e-05, 4.53186384e-05,
       4.62606376e-05, 1.36249788e-05, 1.95972842e-05, 2.26021245e-05,
       2.41052169e-05, 2.47404922e-05])

In [21]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# section_number = np.arange(1, myvlm.n_spanwise+1, 1)

# fig = plt.figure()
# ax1 = fig.add_subplot(111)

# #ax1.plot(cl[0], section_number, label='jib')
# ax1.plot(cl[0],section_number, label='main')
# plt.ylabel('section number')
# plt.xlabel('section cl')
# plt.legend(loc='upper left')
# plt.show()